# Exp S2g: High Noise Regime (η=0.8)

## 目的
高ノイズ環境(η=0.8)での相転移を確認。

## 実験設計
- **η**: 0.8（高ノイズ）
- **λ**: 0.40, 0.44, 0.48（R3aでλ_↑≈0.44と判明）
- **Seeds**: 各34 (合計102 runs)

## 推定時間
~102 × 11 min ≈ **19h**

## 科学的目標
- R3aでη=0.8ではλ_↑=0.44（全seed同一）と判明
- η=0.4(λ_c≈0.50)との比較: 高ノイズで遷移点が劇的に低下
- PRX Figure: η依存性の相図完成

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob, json, time
from datetime import datetime

EXP_NAME = 'exp_S2_deep_bimodality'
NOTEBOOK_ID = 'S2g'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Resuming: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Notebook: {NOTEBOOK_ID} (η=0.8, high noise regime)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
def get_resnet18():
    model = resnet18(weights=None, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    return model

class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

In [ ]:
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16
NOISE_RATE = 0.8  # HIGH NOISE!
LAMBDA_VALUES = [0.40, 0.44, 0.48]  # Based on R3a: λ_↑ ≈ 0.44 for η=0.8
SEEDS_PER_LAMBDA = 34

experiments = []
for lam in LAMBDA_VALUES:
    for s in range(SEEDS_PER_LAMBDA):
        experiments.append({'lambda': lam, 'seed': s})

print(f'η = {NOISE_RATE} (HIGH NOISE)')
print(f'λ values: {LAMBDA_VALUES}')
print(f'Seeds per λ: {SEEDS_PER_LAMBDA}')
print(f'Total: {len(experiments)} runs')

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    noisy = labels.copy()
    n_noisy = int(noise_rate * len(labels))
    idx = np.random.choice(len(labels), n_noisy, replace=False)
    for i in idx:
        noisy[i] = np.random.choice([l for l in range(10) if l != labels[i]])
    return noisy

def load_cifar10():
    tr = transforms.Compose([transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(),
                             transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    te = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))])
    return torchvision.datasets.CIFAR10('./data', True, tr, download=True), torchvision.datasets.CIFAR10('./data', False, te, download=True)

def evaluate(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            correct += (model(x).argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

In [ ]:
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    opt = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    sched = optim.lr_scheduler.MultiStepLR(opt, [50, 75], 0.1)
    crit = nn.CrossEntropyLoss()
    clean_t = torch.tensor(clean_labels, device=device)
    noisy_t = torch.tensor(noisy_labels, device=device)
    cached_gv = None
    step = 0
    cos_hist = []
    error_hist = []
    
    for ep in range(EPOCHS):
        model.train()
        ep_cos = []
        for x, _, idx in train_loader:
            x, idx = x.to(device), idx.to(device)
            bn, bc = noisy_t[idx], clean_t[idx]
            
            opt.zero_grad()
            loss_s = crit(model(x), bn)
            loss_s.backward(retain_graph=True)
            gs = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            if step % K == 0 or cached_gv is None:
                opt.zero_grad()
                loss_v = crit(model(x), bc)
                loss_v.backward()
                cached_gv = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            gs_n = gs / (gs.norm() + 1e-12)
            gv_n = cached_gv / (cached_gv.norm() + 1e-12)
            ep_cos.append((gs_n @ gv_n).item())
            
            g_mix = (1 - lam) * gs_n + lam * gv_n
            opt.zero_grad()
            i = 0
            for p in model.parameters():
                n = p.numel()
                p.grad = g_mix[i:i+n].view(p.shape).clone()
                i += n
            opt.step()
            step += 1
        sched.step()
        cos_hist.append(np.mean(ep_cos))
        
        if (ep + 1) % 10 == 0:
            error_hist.append({'epoch': ep + 1, 'error': 1 - evaluate(model, test_loader)})
    
    final_error = 1 - evaluate(model, test_loader)
    return final_error, np.mean(cos_hist), error_hist

In [ ]:
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader = DataLoader(IndexedDataset(trainset), BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

m = get_resnet18().to(device)
for _ in range(20): _ = m(torch.randn(BATCH_SIZE,3,32,32,device=device))
del m; torch.cuda.empty_cache()
print('Ready')

In [ ]:
results = []
ckpt = f'{SAVE_DIR}/{NOTEBOOK_ID}_checkpoint.json'
done = set()

if os.path.exists(ckpt):
    results = json.load(open(ckpt))
    done = {(r['lambda'], r['seed']) for r in results}
    print(f'Loaded: {len(done)} done')

total = len(experiments)
for i, exp in enumerate(experiments):
    lam, seed = exp['lambda'], exp['seed']
    if (lam, seed) in done: continue
    
    run = len(results) + 1
    print(f'\n[{run}/{total}] η={NOISE_RATE} λ={lam} seed={seed}')
    
    set_seed(seed)
    noisy = inject_label_noise(clean_labels, NOISE_RATE, seed)
    model = get_resnet18().to(device)
    
    t0 = time.time()
    err, cos, error_hist = train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy, lam)
    dt = time.time() - t0
    
    phase = 'ordered' if err <= 0.20 else ('collapsed' if err >= 0.40 else 'intermediate')
    results.append({
        'experiment_id': f'{NOTEBOOK_ID}-{lam:.2f}-{seed:03d}',
        'eta': NOISE_RATE,
        'lambda': lam,
        'seed': seed,
        'final_error': err,
        'avg_cos': cos,
        'phase': phase,
        'time': dt,
        'error_history': error_hist
    })
    done.add((lam, seed))
    
    st = '✅' if phase == 'ordered' else ('⚠️' if phase == 'collapsed' else '🔶')
    print(f'  Error: {err:.4f} | {phase} {st} | {dt/60:.1f}min')
    
    json.dump(results, open(ckpt, 'w'), indent=2)
    remaining = total - run
    print(f'  ETA: {remaining*dt/3600:.1f}h')
    
    del model; torch.cuda.empty_cache()

print('\n' + '='*50 + f'\n{NOTEBOOK_ID} DONE\n' + '='*50)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Save results
json.dump(results, open(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.json', 'w'), indent=2)
df = pd.DataFrame([{k: v for k, v in r.items() if k != 'error_history'} for r in results])
df.to_csv(f'{SAVE_DIR}/{NOTEBOOK_ID}_results.csv', index=False)

print('='*60)
print(f'{NOTEBOOK_ID} SUMMARY - HIGH NOISE REGIME (η={NOISE_RATE})')
print('='*60)

for lam in LAMBDA_VALUES:
    sub = df[df['lambda'] == lam]
    errors = sub['final_error'].values
    n_ord = (sub['phase'] == 'ordered').sum()
    n_col = (sub['phase'] == 'collapsed').sum()
    n_int = (sub['phase'] == 'intermediate').sum()
    
    print(f'\nλ = {lam}:')
    print(f'  N = {len(sub)}')
    print(f'  Mean error: {errors.mean():.4f} ± {errors.std():.4f}')
    print(f'  Ordered: {n_ord} ({100*n_ord/len(sub):.1f}%)')
    print(f'  Intermediate: {n_int} ({100*n_int/len(sub):.1f}%)')
    print(f'  Collapsed: {n_col} ({100*n_col/len(sub):.1f}%)')

In [ ]:
# Visualization for high noise regime
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors = {0.40: 'green', 0.44: 'orange', 0.48: 'red'}

# Histogram by λ
ax = axes[0]
for lam in LAMBDA_VALUES:
    sub = df[df['lambda'] == lam]
    ax.hist(sub['final_error'], bins=15, alpha=0.5, label=f'λ={lam}', color=colors[lam], edgecolor='black')
ax.axvline(0.20, color='green', linestyle='--', linewidth=2)
ax.axvline(0.40, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Final Error', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title(f'Error Distribution (η={NOISE_RATE})', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3)

# Phase fraction by λ
ax = axes[1]
x = np.arange(len(LAMBDA_VALUES))
width = 0.25

ord_fracs = []
int_fracs = []
col_fracs = []
for lam in LAMBDA_VALUES:
    sub = df[df['lambda'] == lam]
    n = len(sub)
    ord_fracs.append((sub['phase'] == 'ordered').sum() / n * 100 if n > 0 else 0)
    int_fracs.append((sub['phase'] == 'intermediate').sum() / n * 100 if n > 0 else 0)
    col_fracs.append((sub['phase'] == 'collapsed').sum() / n * 100 if n > 0 else 0)

ax.bar(x - width, ord_fracs, width, label='Ordered', color='green', alpha=0.7)
ax.bar(x, int_fracs, width, label='Intermediate', color='orange', alpha=0.7)
ax.bar(x + width, col_fracs, width, label='Collapsed', color='red', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels([f'λ={l}' for l in LAMBDA_VALUES])
ax.set_ylabel('Fraction (%)', fontsize=12)
ax.set_title(f'Phase Fraction (η={NOISE_RATE})', fontsize=12)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Comparison with η=0.4 expectation
ax = axes[2]
mean_errors = [df[df['lambda'] == lam]['final_error'].mean() for lam in LAMBDA_VALUES]
std_errors = [df[df['lambda'] == lam]['final_error'].std() for lam in LAMBDA_VALUES]
ax.errorbar(LAMBDA_VALUES, mean_errors, yerr=std_errors, fmt='o-', capsize=5, 
            color='purple', linewidth=2, markersize=8, label=f'η={NOISE_RATE}')
ax.axhline(0.20, color='green', linestyle='--', linewidth=2, label='ordered threshold')
ax.axhline(0.40, color='red', linestyle='--', linewidth=2, label='collapse threshold')
ax.axvline(0.44, color='orange', linestyle=':', linewidth=2, label='R3a λ_↑=0.44')
ax.set_xlabel('λ', fontsize=12)
ax.set_ylabel('Mean Error', fontsize=12)
ax.set_title(f'Transition Curve (η={NOISE_RATE})', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{SAVE_DIR}/figures/{NOTEBOOK_ID}_high_noise.png', dpi=150)
plt.show()

print('\n📊 Key finding for PRX Intelligence:')
print(f'  η=0.4: λ_c ≈ 0.50 (from S1/S2a-e)')
print(f'  η=0.8: λ_c ≈ 0.44 (from R3a, confirmed here)')
print(f'  → Noise rate η controls transition point λ_c')